# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [40]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [41]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))


In [42]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get transfers and trades

In [43]:
df_trade = pd.read_sql_query('SELECT * FROM trades', con)
df_transfer = pd.read_sql_query('SELECT * FROM transfers', con)


In [44]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [45]:
df_trade_summ

,asset,feesPaid
0,AAVE,5094.591762
1,APE,30758.247973
2,ATOM,28396.377724
3,AUD,81.850207
4,AVAX,42987.663092
5,AXS,2664.839332
6,BNB,50876.626429
7,DOGE,10437.786591
8,DYDX,35656.608562
9,EUR,2142.781228


## Get the market data

In [47]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [48]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
df_markets

,asset,marketDebt
0,sETH,8.108333e+06
1,sBTC,8.254472e+06
2,LINK,3.797184e+05
3,SOL,6.703578e+05
4,AVAX,3.215581e+05
5,AAVE,2.201287e+03
6,UNI,1.011234e+05
7,MATIC,4.501527e+05
8,APE,6.550199e+04
9,DYDX,3.924243e+04


In [49]:
df_markets.dtypes

asset          object
marketDebt    float64
dtype: object

In [50]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,8.108333e+06,8.868274e+06,932899.404332,-1.692840e+06
1,sBTC,8.254472e+06,8.625212e+06,793610.582846,-1.164351e+06
2,LINK,3.797184e+05,4.669901e+05,92503.661402,-1.797753e+05
3,SOL,6.703578e+05,7.350607e+05,111622.346456,-1.763252e+05
4,AVAX,3.215581e+05,2.511730e+05,42987.663092,2.739750e+04
5,AAVE,2.201287e+03,3.721074e+02,5094.591762,-3.265412e+03
6,UNI,1.011234e+05,1.152431e+05,10702.703136,-2.482233e+04
7,MATIC,4.501527e+05,5.444329e+05,87282.571105,-1.815627e+05
8,APE,6.550199e+04,3.571943e+04,30758.247973,-9.756797e+02
9,DYDX,3.924243e+04,5.921073e+04,35656.608562,-5.562491e+04


In [51]:
df_pnl.to_csv('output/market_upnl.csv', index=False)

In [52]:
df_pnl['netPnl'].sum()

-4014774.990959737

In [53]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1692839.83
1,sBTC,-1164350.77
11,OP,-292177.78
7,MATIC,-181562.71
2,LINK,-179775.29
3,SOL,-176325.23
19,FTM,-128014.66
10,BNB,-57564.33
9,DYDX,-55624.91
16,ATOM,-46678.69
